In [ ]:
# import all packages
import scipy
import pandas
import scipy.optimize
import scipy.stats
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# read in some data we created for this example (.dat is a generic filename, it's just a text file)
data_filename='https://raw.githubusercontent.com/uofscphysics/STEM_Python_Course/Summer2020/02_Week2/Data/1D_intro_examples.dat'
example_data_1D = pandas.read_csv(data_filename,sep=',',header=0)#this file is separated by spaces and its first line contains the names of the columns (header) 
print(example_data_1D.head())

In [ ]:
#Let's plot the data, with error bars, that we read from file (See Day 2)
plt.errorbar(example_data_1D['x'], #x,y,and error are the column names
             example_data_1D['y'], 
             yerr=example_data_1D['error'],#yerr denotes an error in the y-direction for plotting
             fmt='.') #fmt is "format", saying that I want data marked by "points"
plt.xlabel('x') #set the x-axis label 
plt.ylabel('y') #set the y-axis label
plt.show()

In [ ]:
a_true=1.4
b_true=.4
c_true=2.5


In [ ]:
#The data were generated with a simple quadratic equation:
#ax^2+bx+c. 
def my_model(x,a,b,c): #We define the model described above
    return(a*x**2+b*x+c)


### Write a $\chi$-square function that we can then minimize. The $\chi$-square statistic is defined as the sum of the square of the ratio of the difference between the model and data (observed-model) and the uncertainty. Here's some pseudo-code:

`chisq = sum( ((obs-model)/err)^2)`

### Make sure your function matches the format required by the `scipy.optimize.minimize` function (It accepts a single argument, which is a tuple of items passed by the call to minimize). 

In [ ]:
#Let's try and fit the data with a chi-square minimization
def chisq_likelihood(theta, args):
    #This function accepts an argument "theta", which is 
    #a list of model parameters a, b and c. It then calculates
    #a chi-square statistic that it returns, which compares
    #the observations, errors, and model provided in args.
    
    x, y, yerr,mod = args #args is a list, so this is the same as x=args[0],y=args[1],yerr=args[2]. x,y, and yerr are numpy arrays, mod is a function.
    a,b,c = theta #theta is also a list, so it follows the same as args above
    model_observations = mod(x,a,b,c) #mod (a model) is the 4th element of args, and it accepts x values, and the three model parameters a,b,c. Now model_observations contains the model values at every point in x (and is a numpy array)
    inv_sigma2 = 1./yerr**2 #The chi-square statistic contains an inverse-square error, which we calculate here
    chisquare = np.sum((y-model_observations)**2*inv_sigma2 )#calculate the chi-square statistic. 
    return chisquare



### For this exercise we are assuming that the best-fit parameters for the data are found where the $\chi$-square is at a minimum, which is essentially the definition of the statistic. So now use the `scipy.optimize.minimize` function to "minimize" your $\chi$-square function. 

In [ ]:
#Use scipy.optimize.minimize to minimize the chi-square, which
#is the same as maximizing the likelihood.
result = scipy.optimize.minimize(chisq_likelihood, #the first argument is the function to minimize, which must accept a list of parameters as its first argument, and any other necessary agruments as its second.
                                 x0=[1,1,1], #x0 is a first guess for each of your parameters (depends on your model/data)
                                 bounds=[(-100,100),(-100,100),(-100,100)], #optional bounds for each of your parameters
                                 args=[example_data_1D['x'],#the args passed to chisq_likelihood above
                                       example_data_1D['y'],
                                       example_data_1D['error'],
                                       my_model])#my_model is defined in a previous cell

print(result)#scipy.optimize.minimize returns a dictionary which I've called result. It has other information, the "x" element has the best fit values


### Plot the data with your best-fit result as a line through it. How does it look? The best-fit parameters from the result of a `scipy.optimize.minimize` function can be found in the following way:

`result=scipy.optimize.minimize(...)`
`bestFitParameters=result.x`

In [ ]:
a_ml,b_ml,c_ml = result["x"] #get our best fit values from result

#set up plotting the model over the data
plt.errorbar(example_data_1D['x'],
             example_data_1D['y'],
             yerr=example_data_1D['error'],
             fmt='.',
             label='Data')

plt.plot(example_data_1D['x'],
         my_model(example_data_1D['x'],a_ml,b_ml,c_ml),
         'g--',#make the line green and dashed
         label='$\chi^2$ Fit')

plt.xlabel('x')
plt.ylabel('y')
plt.legend()
print('Chi-square fit:',a_ml, b_ml,c_ml)
print('True:',a_true,b_true,c_true)
plt.show()